# EDA Dataset

## Explore

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
list1 =np.array(range(10))
list1

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [3]:
df = pd.read_csv('Data/ebayShort.csv', index_col=0)

In [4]:
df.head(100) 

,b2c_c2c,seller_id,declared_handling_days,acceptance_scan_timestamp,shipment_method_id,shipping_fee,carrier_min_estimate,carrier_max_estimate,item_zip,buyer_zip,category_id,item_price,quantity,payment_datetime,delivery_date,weight,weight_units,package_size,record_number
0,B2C,25454,3.0,2019-03-26 15:11:00.000-07:00,0,0.0,3,5,97219,49040,13,27.95,1,2019-03-24 03:56:49.000-07:00,2019-03-29,5,1,LETTER,1
1,C2C,6727381,2.0,2018-06-02 12:53:00.000-07:00,0,3.0,3,5,11415-3528,62521,0,20.50,1,2018-06-01 13:43:54.000-07:00,2018-06-05,0,1,PACKAGE_THICK_ENVELOPE,2
2,B2C,18507,1.0,2019-01-07 16:22:00.000-05:00,0,4.5,3,5,27292,53010,1,19.90,1,2019-01-06 00:02:00.000-05:00,2019-01-10,9,1,PACKAGE_THICK_ENVELOPE,3
3,B2C,4677,1.0,2018-12-17 16:56:00.000-08:00,0,0.0,3,5,90703,80022,1,35.50,1,2018-12-16 10:28:28.000-08:00,2018-12-21,8,1,PACKAGE_THICK_ENVELOPE,4
4,B2C,4677,1.0,2018-07-27 16:48:00.000-07:00,0,0.0,3,5,90703,55070,1,25.00,1,2018-07-26 18:20:02.000-07:00,2018-07-30,3,1,PACKAGE_THICK_ENVELOPE,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,B2C,4749,0.0,2019-10-23 12:51:00.000-06:00,0,0.0,3,5,84093,92154,17,7.50,1,2019-10-22 13:19:46.000-06:00,2019-10-26,48,1,PACKAGE_THICK_ENVELOPE,96
96,B2C,3137,1.0,2018-07-16 17:07:00.000-07:00,0,0.0,3,5,92807,38242,12,8.49,1,2018-07-15 19:00:23.000-07:00,2018-07-20,2,1,PACKAGE_THICK_ENVELOPE,97
97,B2C,3137,1.0,2018-09-19 16:56:00.000-07:00,0,0.0,3,5,92807,59901,12,8.49,1,2018-09-18 20:25:25.000-07:00,2018-09-22,2,1,PACKAGE_THICK_ENVELOPE,98
98,B2C,7482,1.0,2019-03-04 15:30:00.000-05:00,1,0.0,2,5,49548,94507,13,50.59,1,2019-03-04 15:08:20.000-05:00,2019-03-06,36,1,PACKAGE_THICK_ENVELOPE,99


In [5]:
df['buyer_zip'].isnull().sum()

0

In [6]:
df['declared_handling_days'].isnull().sum()

47180

In [7]:
df[df['carrier_min_estimate']<0].count()

b2c_c2c                      74
seller_id                    74
declared_handling_days       70
acceptance_scan_timestamp    74
shipment_method_id           74
shipping_fee                 74
carrier_min_estimate         74
carrier_max_estimate         74
item_zip                     74
buyer_zip                    74
category_id                  74
item_price                   74
quantity                     74
payment_datetime             74
delivery_date                74
weight                       74
weight_units                 74
package_size                 74
record_number                74
dtype: int64

In [8]:
df[df['carrier_max_estimate']<0].count()

b2c_c2c                      74
seller_id                    74
declared_handling_days       70
acceptance_scan_timestamp    74
shipment_method_id           74
shipping_fee                 74
carrier_min_estimate         74
carrier_max_estimate         74
item_zip                     74
buyer_zip                    74
category_id                  74
item_price                   74
quantity                     74
payment_datetime             74
delivery_date                74
weight                       74
weight_units                 74
package_size                 74
record_number                74
dtype: int64

In [9]:
df['item_zip'].isnull().sum()

0

In [10]:
df[df['category_id']<0]

,b2c_c2c,seller_id,declared_handling_days,acceptance_scan_timestamp,shipment_method_id,shipping_fee,carrier_min_estimate,carrier_max_estimate,item_zip,buyer_zip,category_id,item_price,quantity,payment_datetime,delivery_date,weight,weight_units,package_size,record_number


In [11]:
df[df['item_price']<0]

,b2c_c2c,seller_id,declared_handling_days,acceptance_scan_timestamp,shipment_method_id,shipping_fee,carrier_min_estimate,carrier_max_estimate,item_zip,buyer_zip,category_id,item_price,quantity,payment_datetime,delivery_date,weight,weight_units,package_size,record_number


In [12]:
#quantity
df['quantity'].unique()

array([  1,   2,   3,   4,   5,   6,   7,   8,  19,  20,  10,  21,  15,
        12,  18,  50,  17,  24,  13,  16,  14,   9,  40,  30,  11,  29,
        25,  38, 100,  45,  60,  56,  52,  44,  41,  36,  75,  22, 200,
       150,  43,  23,  70,  37,  32,  28,  26,  27,  39,  65,  68, 600,
        74,  35,  95,  80, 107,  61])

In [13]:
#Weight_units
df['weight_units'].value_counts()

1    999997
2         3
Name: weight_units, dtype: int64

In [14]:
#weight 
df[df['weight']<0]

,b2c_c2c,seller_id,declared_handling_days,acceptance_scan_timestamp,shipment_method_id,shipping_fee,carrier_min_estimate,carrier_max_estimate,item_zip,buyer_zip,category_id,item_price,quantity,payment_datetime,delivery_date,weight,weight_units,package_size,record_number


In [15]:
df['package_size'].value_counts()

PACKAGE_THICK_ENVELOPE    843381
NONE                       70637
LETTER                     60576
LARGE_ENVELOPE             13874
LARGE_PACKAGE              11532
Name: package_size, dtype: int64

### Some row has package size is NONE. 

In [16]:
df['b2c_c2c'].value_counts()

B2C    707485
C2C    292515
Name: b2c_c2c, dtype: int64

In [17]:
df['declared_handling_days'].value_counts()

1.0     520870
2.0     172839
3.0     162060
0.0      67824
5.0      14854
4.0       8082
10.0      4081
15.0       897
30.0       762
20.0       544
40.0         7
Name: declared_handling_days, dtype: int64

#### some record has the declared_handling_days is 0

# convert date type 

In [18]:
df['acceptance_scan_timestamp']= pd.to_datetime(df['acceptance_scan_timestamp'])

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000000 entries, 0 to 999999
Data columns (total 19 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   b2c_c2c                    1000000 non-null  object 
 1   seller_id                  1000000 non-null  int64  
 2   declared_handling_days     952820 non-null   float64
 3   acceptance_scan_timestamp  1000000 non-null  object 
 4   shipment_method_id         1000000 non-null  int64  
 5   shipping_fee               1000000 non-null  float64
 6   carrier_min_estimate       1000000 non-null  int64  
 7   carrier_max_estimate       1000000 non-null  int64  
 8   item_zip                   1000000 non-null  object 
 9   buyer_zip                  1000000 non-null  object 
 10  category_id                1000000 non-null  int64  
 11  item_price                 1000000 non-null  float64
 12  quantity                   1000000 non-null  int64  
 13  payment_datet

In [23]:
df['payment_datetime']= pd.to_datetime(df['payment_datetime'])

In [24]:
df['delivery_date']= pd.to_datetime(df['delivery_date'])

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000000 entries, 0 to 999999
Data columns (total 19 columns):
 #   Column                     Non-Null Count    Dtype         
---  ------                     --------------    -----         
 0   b2c_c2c                    1000000 non-null  object        
 1   seller_id                  1000000 non-null  int64         
 2   declared_handling_days     952820 non-null   float64       
 3   acceptance_scan_timestamp  1000000 non-null  object        
 4   shipment_method_id         1000000 non-null  int64         
 5   shipping_fee               1000000 non-null  float64       
 6   carrier_min_estimate       1000000 non-null  int64         
 7   carrier_max_estimate       1000000 non-null  int64         
 8   item_zip                   1000000 non-null  object        
 9   buyer_zip                  1000000 non-null  object        
 10  category_id                1000000 non-null  int64         
 11  item_price                 1000000 non

# Create some dummies 

In [26]:
df= pd.get_dummies(df, columns=['b2c_c2c', 'package_size'], drop_first=True)


In [39]:
df.head().T

,0,1,2,3,4
seller_id,25454,6727381,18507,4677,4677
declared_handling_days,3.0,2.0,1.0,1.0,1.0
acceptance_scan_timestamp,2019-03-26 15:11:00-07:00,2018-06-02 12:53:00-07:00,2019-01-07 16:22:00-05:00,2018-12-17 16:56:00-08:00,2018-07-27 16:48:00-07:00
shipment_method_id,0,0,0,0,0
shipping_fee,0.0,3.0,4.5,0.0,0.0
carrier_min_estimate,3,3,3,3,3
carrier_max_estimate,5,5,5,5,5
item_zip,97219,11415-3528,27292,90703,90703
buyer_zip,49040,62521,53010,80022,55070
category_id,13,0,1,1,1


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000000 entries, 0 to 999999
Data columns (total 22 columns):
 #   Column                               Non-Null Count    Dtype         
---  ------                               --------------    -----         
 0   seller_id                            1000000 non-null  int64         
 1   declared_handling_days               952820 non-null   float64       
 2   acceptance_scan_timestamp            1000000 non-null  object        
 3   shipment_method_id                   1000000 non-null  int64         
 4   shipping_fee                         1000000 non-null  float64       
 5   carrier_min_estimate                 1000000 non-null  int64         
 6   carrier_max_estimate                 1000000 non-null  int64         
 7   item_zip                             1000000 non-null  object        
 8   buyer_zip                            1000000 non-null  object        
 9   category_id                          1000000 non-null  int

In [29]:
df.isnull().sum()

seller_id                                  0
declared_handling_days                 47180
acceptance_scan_timestamp                  0
shipment_method_id                         0
shipping_fee                               0
carrier_min_estimate                       0
carrier_max_estimate                       0
item_zip                                   0
buyer_zip                                  0
category_id                                0
item_price                                 0
quantity                                   0
payment_datetime                           0
delivery_date                              0
weight                                     0
weight_units                               0
record_number                              0
b2c_c2c_C2C                                0
package_size_LARGE_PACKAGE                 0
package_size_LETTER                        0
package_size_NONE                          0
package_size_PACKAGE_THICK_ENVELOPE        0
dtype: int

In [38]:
df.info

<bound method DataFrame.info of         seller_id  declared_handling_days  acceptance_scan_timestamp  \
0           25454                     3.0  2019-03-26 15:11:00-07:00   
1         6727381                     2.0  2018-06-02 12:53:00-07:00   
2           18507                     1.0  2019-01-07 16:22:00-05:00   
3            4677                     1.0  2018-12-17 16:56:00-08:00   
4            4677                     1.0  2018-07-27 16:48:00-07:00   
...           ...                     ...                        ...   
999995      23011                     1.0  2018-10-11 14:08:00-04:00   
999996        412                     1.0  2018-11-13 19:57:00-05:00   
999997        412                     1.0  2019-04-01 17:35:00-04:00   
999998       8303                     3.0  2018-10-03 08:09:00-04:00   
999999        882                     3.0  2018-09-25 09:22:11-07:00   

        shipment_method_id  shipping_fee  carrier_min_estimate  \
0                        0          0